In [1]:
from pyspark.sql.functions import col, when, sum, coalesce, lit, concat_ws
from pyspark.sql.functions import udf
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, LongType, FloatType, MapType, ArrayType
import re
import demoji

In [2]:
from pyspark.sql import SparkSession

# Initialize Spark session
spark = SparkSession.builder \
    .appName("Read JSON Example") \
    .config("spark.driver.memory", "15g") \
    .master("local[*]") \
    .getOrCreate()

ERROR StatusLogger Reconfiguration failed: No configuration found for '5ffd2b27' at 'null' in 'null'
ERROR StatusLogger Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'
25/04/04 12:24:21 WARN Utils: Your hostname, Mahaveers-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.9 instead (on interface en0)
25/04/04 12:24:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/Library/Spark3.5/spark-3.5.4-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/mahaveerchauhan/.ivy2/cache
The jars for the packages stored in: /Users/mahaveerchauhan/.ivy2/jars
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f06c001c-06b3-40c1-bda5-1ae33a4c1e33;1.0
	confs: [default]
	found org.apache.spark#spark-avro_2.12;3.5.4 in central
	found org.tukaani#xz;1.9 in local-m2-cache
:: resolution report :: resolve 84ms :: artifacts dl 2ms
	:: modules in use:
	org.apache.spark#spark-avro_2.12;3.5.4 from central in [default]
	org.tukaani#xz;1.9 from local-m2-cache in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   0   |   0   |   0   ||   2   |   0   |
	-----------------------------------

In [3]:
review_schema = StructType([
    StructField("rating", FloatType(), True),
    StructField("title", StringType(), True),
    StructField("text", StringType(), True),
    StructField("images", ArrayType(StringType()), True),
    StructField("asin", StringType(), True),
    StructField("parent_asin", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("timestamp", LongType(), True),
    StructField("helpful_vote", IntegerType(), True),
    StructField("verified_purchase", BooleanType(), True)
])

In [4]:
review_df = spark.read.schema(review_schema).json("./Amazon_Review_Dataset/Cell_Phones_and_Accessories/Cell_Phones_and_Accessories.jsonl")
review_df.show()

+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+------------+-----------------+
|rating|               title|                text|              images|      asin|parent_asin|             user_id|    timestamp|helpful_vote|verified_purchase|
+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+------------+-----------------+
|   4.0|No white backgrou...|I bought this bc ...|[{"small_image_ur...|B08L6L3X1S| B08L6L3X1S|AFKZENTNBQ7A7V7UX...|1612044451196|           0|             true|
|   5.0|Awesome!  Great p...|Perfect. How piss...|                  []|B079BPGF6C| B079BPGF6C|AFKZENTNBQ7A7V7UX...|1534443517349|           2|             true|
|   5.0|Worked but took a...|Overall very happ...|[{"small_image_ur...|B088DR7Z5B| B0BBGGC8F2|AGCI7FAH4GL5FI65H...|1629235304798|           3|             true|
|   4.0|              Decent|Laste

In [5]:
print("Number of rows:", review_df.count())

Number of rows: 20812945


In [6]:
null_counts = review_df.select([
    (when(col(c).isNull(), 1).otherwise(0)).alias(c) for c in review_df.columns
])

null_counts_summary = null_counts.agg(*[sum(col(c)).alias(c) for c in null_counts.columns])

null_counts_summary.show()

+------+-----+----+------+----+-----------+-------+---------+------------+-----------------+
|rating|title|text|images|asin|parent_asin|user_id|timestamp|helpful_vote|verified_purchase|
+------+-----+----+------+----+-----------+-------+---------+------------+-----------------+
|     0|    0|   0|     0|   0|          0|      0|        0|           0|                0|
+------+-----+----+------+----+-----------+-------+---------+------------+-----------------+



In [ ]:
review_df = review_df.dropna()

review_df.show()

+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+------------+-----------------+
|rating|               title|                text|              images|      asin|parent_asin|             user_id|    timestamp|helpful_vote|verified_purchase|
+------+--------------------+--------------------+--------------------+----------+-----------+--------------------+-------------+------------+-----------------+
|   4.0|No white backgrou...|I bought this bc ...|[{"small_image_ur...|B08L6L3X1S| B08L6L3X1S|AFKZENTNBQ7A7V7UX...|1612044451196|           0|             true|
|   5.0|Awesome!  Great p...|Perfect. How piss...|                  []|B079BPGF6C| B079BPGF6C|AFKZENTNBQ7A7V7UX...|1534443517349|           2|             true|
|   5.0|Worked but took a...|Overall very happ...|[{"small_image_ur...|B088DR7Z5B| B0BBGGC8F2|AGCI7FAH4GL5FI65H...|1629235304798|           3|             true|
|   4.0|              Decent|Laste

In [ ]:
filtered_review_df = review_df.select("user_id", "asin", "parent_asin", "title", "text", "rating", "verified_purchase", "timestamp")

filtered_review_df.show()

+--------------------+----------+-----------+--------------------+--------------------+------+-----------------+-------------+
|             user_id|      asin|parent_asin|               title|                text|rating|verified_purchase|    timestamp|
+--------------------+----------+-----------+--------------------+--------------------+------+-----------------+-------------+
|AFKZENTNBQ7A7V7UX...|B08L6L3X1S| B08L6L3X1S|No white backgrou...|I bought this bc ...|   4.0|             true|1612044451196|
|AFKZENTNBQ7A7V7UX...|B079BPGF6C| B079BPGF6C|Awesome!  Great p...|Perfect. How piss...|   5.0|             true|1534443517349|
|AGCI7FAH4GL5FI65H...|B088DR7Z5B| B0BBGGC8F2|Worked but took a...|Overall very happ...|   5.0|             true|1629235304798|
|AGCI7FAH4GL5FI65H...|B07XRDHDNQ| B07XRDHDNQ|              Decent|Lasted about 9 mo...|   4.0|             true|1590470082910|
|AGCI7FAH4GL5FI65H...|B00A8ZDL9Y| B00A8ZDL9Y|            LOVE IT!|LOVE THIS CASE! W...|   5.0|             true

In [9]:
print("Number of rows:", filtered_review_df.count())

Number of rows: 20812945


In [ ]:
filtered_review_df = filtered_review_df.filter(filtered_review_df["verified_purchase"] == True)

filtered_review_df.show()

+--------------------+----------+-----------+--------------------+--------------------+------+-----------------+-------------+
|             user_id|      asin|parent_asin|               title|                text|rating|verified_purchase|    timestamp|
+--------------------+----------+-----------+--------------------+--------------------+------+-----------------+-------------+
|AFKZENTNBQ7A7V7UX...|B08L6L3X1S| B08L6L3X1S|No white backgrou...|I bought this bc ...|   4.0|             true|1612044451196|
|AFKZENTNBQ7A7V7UX...|B079BPGF6C| B079BPGF6C|Awesome!  Great p...|Perfect. How piss...|   5.0|             true|1534443517349|
|AGCI7FAH4GL5FI65H...|B088DR7Z5B| B0BBGGC8F2|Worked but took a...|Overall very happ...|   5.0|             true|1629235304798|
|AGCI7FAH4GL5FI65H...|B07XRDHDNQ| B07XRDHDNQ|              Decent|Lasted about 9 mo...|   4.0|             true|1590470082910|
|AGCI7FAH4GL5FI65H...|B00A8ZDL9Y| B00A8ZDL9Y|            LOVE IT!|LOVE THIS CASE! W...|   5.0|             true

In [11]:
print("Number of rows:", filtered_review_df.count())

Number of rows: 19726191


In [ ]:
def clean_text(text):
    text = re.sub(r'<br />', ' ', text)
    text = re.sub(r'<.*?>', ' ', text)

    text = demoji.replace(text, '')
    
    text = re.sub(r'[!]+', '!', text)

    text = re.sub(r'[^a-zA-Z0-9\s.!?]', '', text)

    text = text.lower()

    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
clean_text_udf = udf(clean_text, StringType())

filtered_review_df = filtered_review_df.withColumn('text', clean_text_udf(filtered_review_df['text']))
filtered_review_df = filtered_review_df.withColumn('title', clean_text_udf(filtered_review_df['title']))

filtered_review_df.show()

+--------------------+----------+-----------+--------------------+--------------------+------+-----------------+-------------+
|             user_id|      asin|parent_asin|               title|                text|rating|verified_purchase|    timestamp|
+--------------------+----------+-----------+--------------------+--------------------+------+-----------------+-------------+
|AFKZENTNBQ7A7V7UX...|B08L6L3X1S| B08L6L3X1S|no white backgrou...|i bought this bc ...|   4.0|             true|1612044451196|
|AFKZENTNBQ7A7V7UX...|B079BPGF6C| B079BPGF6C|awesome! great pr...|perfect. how piss...|   5.0|             true|1534443517349|
|AGCI7FAH4GL5FI65H...|B088DR7Z5B| B0BBGGC8F2|worked but took a...|overall very happ...|   5.0|             true|1629235304798|
|AGCI7FAH4GL5FI65H...|B07XRDHDNQ| B07XRDHDNQ|              decent|lasted about 9 mo...|   4.0|             true|1590470082910|
|AGCI7FAH4GL5FI65H...|B00A8ZDL9Y| B00A8ZDL9Y|            love it!|love this case! w...|   5.0|             true

# Metadata

In [ ]:
meta_schema = StructType([
    StructField("main_category", StringType(), True),
    StructField("title", StringType(), True),
    StructField("average_rating", FloatType(), True),
    StructField("rating_number", IntegerType(), True),
    StructField("features", ArrayType(StringType()), True),
    StructField("description", ArrayType(StringType()), True),
    StructField("price", StringType(), True),
    StructField("images", StructType([
        StructField("hi_res", ArrayType(StringType()), True),
        StructField("large", ArrayType(StringType()), True),
        StructField("thumb", ArrayType(StringType()), True),
        StructField("variant", ArrayType(StringType()), True)
    ]), True),
    StructField("videos", StructType([
        StructField("title", ArrayType(StringType()), True),
        StructField("url", ArrayType(StringType()), True),
        StructField("user_id", ArrayType(StringType()), True)
    ]), True),
    StructField("store", StringType(), True),
    StructField("categories", ArrayType(StringType()), True),
    StructField("details", MapType(StringType(), StringType()), True),
    StructField("parent_asin", StringType(), True),
    StructField("bought_together", StringType(), True),
    StructField("subtitle", StringType(), True),
    StructField("author", StringType(), True)
])

In [15]:
meta_df = spark.read.schema(meta_schema).json("./Amazon_Review_Dataset/Cell_Phones_and_Accessories/meta_Cell_Phones_and_Accessories.jsonl")
meta_df.show()

+--------------------+--------------------+--------------+-------------+----------------------+--------------------+-----+------+------+---------------+--------------------+--------------------+-----------+---------------+--------+------+
|       main_category|               title|average_rating|rating_number|              features|         description|price|images|videos|          store|          categories|             details|parent_asin|bought_together|subtitle|author|
+--------------------+--------------------+--------------+-------------+----------------------+--------------------+-----+------+------+---------------+--------------------+--------------------+-----------+---------------+--------+------+
|Cell Phones & Acc...|ARAREE Slim Diary...|           3.8|            5|  [Genuine Cow leat...|[JUST LOOK, You c...| NULL|  NULL|  NULL|         araree|[Cell Phones & Ac...|{Product Dimensio...| B013SK1JTY|           NULL|    NULL|  NULL|
|Cell Phones & Acc...|Bastmei for OnePl...| 

In [ ]:
meta_df = meta_df.select("main_category", "title", "average_rating", "rating_number", 
                         "features", "description", "images", "categories", "parent_asin")

meta_df.show()

+--------------------+--------------------+--------------+-------------+----------------------+--------------------+------+--------------------+-----------+
|       main_category|               title|average_rating|rating_number|              features|         description|images|          categories|parent_asin|
+--------------------+--------------------+--------------+-------------+----------------------+--------------------+------+--------------------+-----------+
|Cell Phones & Acc...|ARAREE Slim Diary...|           3.8|            5|  [Genuine Cow leat...|[JUST LOOK, You c...|  NULL|[Cell Phones & Ac...| B013SK1JTY|
|Cell Phones & Acc...|Bastmei for OnePl...|           4.4|          177|  [Ultra-thin & Ult...|                  []|  NULL|[Cell Phones & Ac...| B07ZPSG8P5|
|Cell Phones & Acc...|Wireless Fones Br...|           4.0|            2|                    []|                  []|  NULL|[Cell Phones & Ac...| B00GKR3L12|
|Cell Phones & Acc...|iPhone 6 Plus + C...|           4.0|

In [ ]:
unique_main_category = meta_df.select("main_category").distinct().limit(1).collect()[0]["main_category"]

meta_df = meta_df.fillna({"main_category": unique_main_category})

In [18]:
print("Number of rows:", meta_df.count())

Number of rows: 1288490


In [ ]:
def merge_text(col_1, col_2):
    col_1 = "" if col_1 is None else col_1
    col_2 = "" if col_2 is None else col_2
    
    text_1 = " ".join(col_1) if isinstance(col_1, list) else col_1
    text_2 = " ".join(col_2) if isinstance(col_2, list) else col_2
    
    raw_text = text_1 + " " + text_2
    
    final_text = clean_text(raw_text)
    
    return final_text

In [ ]:
merge_text_udf = udf(merge_text, StringType())

meta_df = meta_df.withColumn("Text Data", merge_text_udf(meta_df["features"], meta_df["description"]))

meta_df.show()

+--------------------+--------------------+--------------+-------------+----------------------+--------------------+------+--------------------+-----------+--------------------+
|       main_category|               title|average_rating|rating_number|              features|         description|images|          categories|parent_asin|           Text Data|
+--------------------+--------------------+--------------+-------------+----------------------+--------------------+------+--------------------+-----------+--------------------+
|Cell Phones & Acc...|ARAREE Slim Diary...|           3.8|            5|  [Genuine Cow leat...|[JUST LOOK, You c...|  NULL|[Cell Phones & Ac...| B013SK1JTY|genuine cow leath...|
|Cell Phones & Acc...|Bastmei for OnePl...|           4.4|          177|  [Ultra-thin & Ult...|                  []|  NULL|[Cell Phones & Ac...| B07ZPSG8P5|ultrathin ultrali...|
|Cell Phones & Acc...|Wireless Fones Br...|           4.0|            2|                    []|               

In [ ]:
filtered_review_df = filtered_review_df \
    .withColumnRenamed('user_id', 'User ID') \
    .withColumnRenamed('asin', 'ASIN') \
    .withColumnRenamed('parent_asin', 'Parent ASIN') \
    .withColumnRenamed('title', 'Review Title') \
    .withColumnRenamed('text', 'Review Text') \
    .withColumnRenamed('rating', 'Rating') \
    .withColumnRenamed('verified_purchase', 'Verified Purchase')

meta_df = meta_df \
    .withColumnRenamed('main_category', 'Main Category') \
    .withColumnRenamed('title', 'Product Title') \
    .withColumnRenamed('average_rating', 'Average Rating') \
    .withColumnRenamed('rating_number', 'Rating Number') \
    .withColumnRenamed('features', 'Features') \
    .withColumnRenamed('description', 'Descriptions') \
    .withColumnRenamed('images', 'Images') \
    .withColumnRenamed('categories', 'Categories') \
    .withColumnRenamed('parent_asin', 'Parent ASIN')

In [22]:
meta_df = meta_df.drop('Features', 'Descriptions')
meta_df.show()

+--------------------+--------------------+--------------+-------------+------+--------------------+-----------+--------------------+
|       Main Category|       Product Title|Average Rating|Rating Number|Images|          Categories|Parent ASIN|           Text Data|
+--------------------+--------------------+--------------+-------------+------+--------------------+-----------+--------------------+
|Cell Phones & Acc...|ARAREE Slim Diary...|           3.8|            5|  NULL|[Cell Phones & Ac...| B013SK1JTY|genuine cow leath...|
|Cell Phones & Acc...|Bastmei for OnePl...|           4.4|          177|  NULL|[Cell Phones & Ac...| B07ZPSG8P5|ultrathin ultrali...|
|Cell Phones & Acc...|Wireless Fones Br...|           4.0|            2|  NULL|[Cell Phones & Ac...| B00GKR3L12|                    |
|Cell Phones & Acc...|iPhone 6 Plus + C...|           4.0|           15|  NULL|[Cell Phones & Ac...| B00PB8U8BW|slimfit design fo...|
|Cell Phones & Acc...|Case for Galaxy S...|           4.0|    

In [23]:
filtered_review_df.show()

+--------------------+----------+-----------+--------------------+--------------------+------+-----------------+-------------+
|             User ID|      ASIN|Parent ASIN|        Review Title|         Review Text|Rating|Verified Purchase|    timestamp|
+--------------------+----------+-----------+--------------------+--------------------+------+-----------------+-------------+
|AFKZENTNBQ7A7V7UX...|B08L6L3X1S| B08L6L3X1S|no white backgrou...|i bought this bc ...|   4.0|             true|1612044451196|
|AFKZENTNBQ7A7V7UX...|B079BPGF6C| B079BPGF6C|awesome! great pr...|perfect. how piss...|   5.0|             true|1534443517349|
|AGCI7FAH4GL5FI65H...|B088DR7Z5B| B0BBGGC8F2|worked but took a...|overall very happ...|   5.0|             true|1629235304798|
|AGCI7FAH4GL5FI65H...|B07XRDHDNQ| B07XRDHDNQ|              decent|lasted about 9 mo...|   4.0|             true|1590470082910|
|AGCI7FAH4GL5FI65H...|B00A8ZDL9Y| B00A8ZDL9Y|            love it!|love this case! w...|   5.0|             true

In [ ]:
merged_df = filtered_review_df.join(meta_df, on='Parent ASIN', how='inner')

merged_df.show()

+-----------+--------------------+----------+--------------------+--------------------+------+-----------------+-------------+--------------------+--------------------+--------------+-------------+------+--------------------+--------------------+
|Parent ASIN|             User ID|      ASIN|        Review Title|         Review Text|Rating|Verified Purchase|    timestamp|       Main Category|       Product Title|Average Rating|Rating Number|Images|          Categories|           Text Data|
+-----------+--------------------+----------+--------------------+--------------------+------+-----------------+-------------+--------------------+--------------------+--------------+-------------+------+--------------------+--------------------+
| 159775045X|AHUAOQIWKQCLVGTTS...|159775045X|discontinued product|national geograph...|   2.0|             true|1354206930000|Cell Phones & Acc...|National Geograph...|           4.2|            3|  NULL|                  []|print photo quali...|
| 1783617799

In [25]:
print("Number of rows:", merged_df.count())

Number of rows: 19726191


In [26]:
print(merged_df.columns)

['Parent ASIN', 'User ID', 'ASIN', 'Review Title', 'Review Text', 'Rating', 'Verified Purchase', 'timestamp', 'Main Category', 'Product Title', 'Average Rating', 'Rating Number', 'Images', 'Categories', 'Text Data']


In [ ]:
merged_df.write.mode("overwrite").parquet("Cell_Phones_and_Accessories.parquet", compression="SNAPPY")